In [8]:
import spacy
import pandas as pd
from spacy import displacy
import re

In [9]:
nlp = spacy.load("en_core_web_sm")

In [56]:
data_frame = (pd.read_csv("C:\\Users\\Sectorea\\Code\\database_linkedin\\etl_consolidated\\data_science_London_Past_24_hours_2020-04-14.csv"))
data_frame.head()

,id,title,company,location,posted,applicants,text,level,type,functions,industries
0,1806342384,Machine Learning Engineer,Paritas Recruitment,"London, England, United Kingdom",0.875000,173,Paritas Recruitment are working with a stealth...,Mid-Senior level,Full-time,Information Technology Engineering Other,Information Technology and Services Computer ...
1,1825361202,"Sales Manager, Advertising",GumGum,"London, England, United Kingdom",0.333333,0,Job DescriptionSummary Our advertising busines...,Entry level,Full-time,Sales Business Development,Marketing and Advertising Online Media Comput...
2,1824823679,IT Support Analyst / Helpdesk / 1st Line Support,AboutYouRecruitment,"London, England, United Kingdom",0.875000,0,IT Support Analyst / Helpdesk / 1st Line Suppo...,Entry level,Full-time,Information Technology,Marketing and Advertising Computer Software S...
3,1825453231,Senior Electronics Design Engineers – Medical ...,Newton Colmore Consulting,"London, England, United Kingdom",0.208333,0,"An established Medical Devices company, based ...",Associate,Full-time,Engineering Information Technology,Medical Devices Research Staffing and Recruiting
4,1825427920,"Data Scientist, Performance Marketing",King,"London, England, United Kingdom",0.208333,0,Job DescriptionSalary: Market RateApplication ...,Entry level,Full-time,Engineering Information Technology,Staffing and Recruiting Human Resources Compu...


In [55]:
text = data_frame.loc[7, 'text']
text

'We’re Sky, Europe’s biggest entertainment brand. Think top-quality shows. Breaking news. Innovative tech. Must-have products. Careers here mean the freedom and support you need to make an impact – pushing boundaries, creating solutions, hitting targets. And as part of our close-knit team, you’ll enjoy plenty of benefits. Plus, experiences you’ll only find at Sky.“Global OTT Capabilities is a brand-new team with an ambitious remit. We’re looking for dynamic, creative people who can build relationships, challenge the way we do things and help us deliver solutions that will scale across all our OTT products. We’re looking forward to welcoming someone new to our friendly team” – Head of DepartmentThe perksSky Q, a generous pension and private health care. Access to over 12,000 LinkedIn Learning courses to support your development. And if that’s not enough, our award-winning Osterley campus boasts six subsidised restaurants, a cinema, gym, and much more.To find out more about working with 

In [51]:
def get_clean_tokens(text):

    # clean_text = re.sub('[!#?¿:"+-]', ' ', text)
    clean_text = re.sub(r"([A-Z])([A-Z])([a-z])", r"\1 \2\3", text)
    clean_text = re.sub(r"([a-z])([A-Z])", r"\1 \2", clean_text)

    return clean_text

In [52]:
clean_text = get_clean_tokens(text)

In [24]:
text = data_frame.loc[row, 'text']
doc = nlp(text)
print([(X.text, X.label_) for X in doc.ents])

[('110,000', 'MONEY'), ('Data Science', 'ORG'), ('Product Owners', 'ORG'), ('Software Engineers', 'ORG'), ('Data Engineers', 'ORG'), ('Ph.D.', 'WORK_OF_ART'), ('Python', 'GPE'), ('Hadoop', 'GPE'), ('Tableau', 'ORG'), ('SQL', 'ORG'), ('Hive Excellent', 'ORG'), ('KEYWORDSPython', 'ORG'), ('SQL', 'ORG'), ('Hadoop', 'GPE'), ('AWS', 'ORG'), ('GCP', 'ORG'), ('Spark', 'GPE'), ('Data Scientist', 'PERSON'), ('Data Science', 'PERSON'), ('Machine Learning', 'GPE'), ('Statistics', 'GPE')]


In [25]:
def get_values(row):
    text = data_frame.loc[row, 'text']
    doc = nlp(text)
    print([(X.text, X.label_) for X in doc.ents])


In [53]:
displacy.render(nlp(clean_text), jupyter=True, style='ent')

In [ ]:
displacy.render(nlp(text), style='dep', jupyter = True, options = {'distance': 120})

In [ ]:
[(x.orth_,x.pos_, x.lemma_) for x in [y 
                                      for y
                                      in nlp(text) 
                                      if not y.is_stop and y.pos_ != 'PUNCT']]

In [ ]:
documents = list(data_frame.text)

In [ ]:
print(len(documents))

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [63]:
data_frame.dropna(inplace=True)
documents = list(data_frame.text)

In [64]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(documents)

In [62]:
X.shape

(683, 14122)

In [ ]:
X

In [ ]:
from sklearn.cluster.bicluster import SpectralCoclustering
from matplotlib import pyplot as plt
import numpy as np
from collections import Counter


In [ ]:
cocluster = SpectralCoclustering(n_clusters=10,
                                 random_state=0)

print("number of movies:", X.shape[0])
print("number of users :", X.shape[1])

In [ ]:
print("Performing the coclustering...")
cocluster.fit(X)
y_cocluster = cocluster.row_labels_

In [ ]:
i = list(y_cocluster)
Counter(y_cocluster)

In [ ]:
cocluster.n_clusters

In [ ]:


cmap = plt.cm.bwr

sorted_rows_inds = np.argsort(cocluster.row_labels_)
sorted_cols_inds = np.argsort(cocluster.column_labels_)

fit_data = X[sorted_rows_inds]
fit_data = fit_data[:, sorted_cols_inds]

In [ ]:
vmax = max([abs(X.min()), abs(X.max())])
vmax = 0.001

In [ ]:
X.shape

In [ ]:
def reduce_X(X, row_factor=50, col_factor=50):
    nrows, ncols = np.shape(X)
#    nrows_red = nrows//row_factor
#    ncols_red = ncols//col_factor
    nrows_red = int(np.ceil(nrows/row_factor))
    ncols_red = int(np.ceil(ncols/col_factor))
    reducida = np.zeros((nrows_red, ncols_red))
    aux_row = 0
    for row in range(nrows_red):
        if row % 10 == 0:
            print("{} / {}".format(row+1, nrows_red))
        aux_col = 0
        for col in range(ncols_red):
            reducida[row,col] = X[aux_row:(aux_row+row_factor),
                                  aux_col:(aux_col+col_factor)].mean()
            aux_col += col_factor
        aux_row += row_factor
    return reducida

In [ ]:
row_factor = 1
col_factor = 10
reducida_X = reduce_X(X, row_factor=row_factor, col_factor=col_factor)
reducida_fit_data = reduce_X(fit_data, row_factor=row_factor, col_factor=col_factor)

In [ ]:
print("X shape reduced from", X.shape, "to", reducida_X.shape)

In [ ]:
cmap = plt.cm.cool
n_clusters = cocluster.n_clusters

plt.figure(figsize=(25,25))
#plt.subplot(1,2,1)
plt.imshow(reducida_X, cmap=cmap, vmin=0, vmax=vmax, aspect=1)
#plt.xticks(range(len(np.array(feature_names)[::col_factor]))[::10],
#           np.array(feature_names)[::col_factor][::10],
#           fontsize=14, rotation=85)
plt.title("original dataset", fontsize=24)
plt.xlabel('Users', fontsize=24)
plt.ylabel('Movies', fontsize=24)
plt.show()

plt.figure(figsize=(25,25))
#plt.subplot(1,2,2)
plt.imshow(reducida_fit_data, cmap=cmap, vmin=0, vmax=vmax, aspect=1)
#plt.xticks(range(len(np.array(feature_names)[::col_factor]))[::10],
#           np.array(feature_names)[sorted_cols_inds][::col_factor][::10],
#           fontsize=14, rotation=85)
plt.title("Dataset after biclustering, rearranged to show biclusters", fontsize=24)
corner_y = -0.5
c = 0
for i in range(n_clusters):
    corner_x = -0.5
    for j in range(n_clusters):
#        sh = cocluster.get_shape(i)
        sh = [cocluster.get_shape(i)[0]/row_factor,
              cocluster.get_shape(j)[1]/col_factor]
        if i==j:
            plt.plot([corner_x, corner_x+sh[1], corner_x+sh[1], corner_x, corner_x],
                     [corner_y, corner_y, corner_y+sh[0], corner_y+sh[0], corner_y], 'g', linewidth=3)
        corner_x += sh[1]
        c += 1
    corner_y += sh[0]
plt.xlabel('Users', fontsize=24)
plt.ylabel('Movies', fontsize=24)
plt.show()